In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [27]:
get_data = pd.read_excel('data/ORG/경락가격/대파/대파 월별 총거래물량(원본).xlsx', index_col=None)

In [9]:
get_data = get_data.sort_index(ascending=False)

In [10]:
seoul_data = get_data[get_data['시장명'].str.contains('서울')]
busan_data = get_data[get_data['시장명'].str.contains('부산')]
daegu_data = get_data[get_data['시장명'].str.contains('대구')]
daejeon_data = get_data[get_data['시장명'].str.contains('대전')]
gwangju_data = get_data[get_data['시장명'].str.contains('광주')]

In [14]:
seoul_date = seoul_data['DATE'].unique()
print(seoul_date)
busan_date = busan_data['DATE'].unique()
print(busan_date)
daegu_date = daegu_data['DATE'].unique()
print(daegu_date)
daejeon_date = daejeon_data['DATE'].unique()
print(daejeon_date)
gwangju_date = gwangju_data['DATE'].unique()
print(gwangju_date)

['2020-01' '2020-02' '2020-03' '2020-04' '2020-05' '2020-06' '2020-07'
 '2020-08' '2020-09' '2020-10' '2020-11' '2020-12' '2021-01' '2021-02'
 '2021-03' '2021-04' '2021-05' '2021-06' '2021-07' '2021-08' '2021-09'
 '2021-10' '2021-11' '2021-12' '2022-01' '2022-02' '2022-03' '2022-04'
 '2022-05' '2022-06' '2022-07' '2022-08' '2022-09' '2022-10' '2022-11'
 '2022-12' '2023-01' '2023-02' '2023-03' '2023-04' '2023-05' '2023-06'
 '2023-07' '2023-08']
['2020-01' '2020-02' '2020-03' '2020-04' '2020-05' '2020-06' '2020-07'
 '2020-08' '2020-09' '2020-10' '2020-11' '2020-12' '2021-01' '2021-02'
 '2021-03' '2021-04' '2021-05' '2021-06' '2021-07' '2021-08' '2021-09'
 '2021-10' '2021-11' '2021-12' '2022-01' '2022-02' '2022-03' '2022-04'
 '2022-05' '2022-06' '2022-07' '2022-08' '2022-09' '2022-10' '2022-11'
 '2022-12' '2023-01' '2023-02' '2023-03' '2023-04' '2023-05' '2023-06'
 '2023-07' '2023-08']
['2020-01' '2020-02' '2020-03' '2020-04' '2020-05' '2020-06' '2020-07'
 '2020-08' '2020-09' '2020-10' '2

In [16]:
all_list = []
for i in seoul_date:
    seoul_sum1 = seoul_data.loc[seoul_data['DATE']==i, '총거래물량(kg)'].values
    seoul_sum2 = seoul_data.loc[seoul_data['DATE']==i, '총거래금액(원)'].values
    all_list.append([i, seoul_sum1.sum(), seoul_sum2.sum(), '서울'])

for i in busan_date:
    busan_sum1 = busan_data.loc[busan_data['DATE']==i, '총거래물량(kg)'].values
    busan_sum2 = busan_data.loc[busan_data['DATE']==i, '총거래금액(원)'].values
    all_list.append([i, busan_sum1.sum(), busan_sum2.sum(), '부산'])

for i in daegu_date:
    daegu_sum1 = daegu_data.loc[daegu_data['DATE']==i, '총거래물량(kg)'].values
    daegu_sum2 = daegu_data.loc[daegu_data['DATE']==i, '총거래금액(원)'].values
    all_list.append([i, daegu_sum1.sum(), daegu_sum2.sum(), '대구'])

for i in daejeon_date:
    daejeon_sum1 = daejeon_data.loc[daejeon_data['DATE']==i, '총거래물량(kg)'].values
    daejeon_sum2 = daejeon_data.loc[daejeon_data['DATE']==i, '총거래금액(원)'].values
    all_list.append([i, daejeon_sum1.sum(), daejeon_sum2.sum(), '대전'])

for i in gwangju_date:
    gwangju_sum1 = gwangju_data.loc[gwangju_data['DATE']==i, '총거래물량(kg)'].values
    gwangju_sum2 = gwangju_data.loc[gwangju_data['DATE']==i, '총거래금액(원)'].values
    all_list.append([i, gwangju_sum1.sum(), gwangju_sum2.sum(), '광주'])

In [17]:
all_list

[['2020-01', 9462928.5, 9136197340, '서울'],
 ['2020-02', 8754610.0, 7387886850, '서울'],
 ['2020-03', 9109415.0, 6478744150, '서울'],
 ['2020-04', 8229718.0, 6756209940, '서울'],
 ['2020-05', 7901230.0, 9477792330, '서울'],
 ['2020-06', 7914290.5, 10485254220, '서울'],
 ['2020-07', 8096983.1, 10570971010, '서울'],
 ['2020-08', 7364979.3, 15393054610, '서울'],
 ['2020-09', 8882357.0, 20153143600, '서울'],
 ['2020-10', 8084234.0, 18937973340, '서울'],
 ['2020-11', 9074613.0, 20620399360, '서울'],
 ['2020-12', 9257988.8, 15235945860, '서울'],
 ['2021-01', 8241883.3, 24222400650, '서울'],
 ['2021-02', 7644953.0, 33632029700, '서울'],
 ['2021-03', 8076544.0, 35220387300, '서울'],
 ['2021-04', 7029457.0, 26378613030, '서울'],
 ['2021-05', 8101083.4, 16244390960, '서울'],
 ['2021-06', 7687400.4, 6819742620, '서울'],
 ['2021-07', 6454223.0, 4956262050, '서울'],
 ['2021-08', 6683397.2, 7546671760, '서울'],
 ['2021-09', 7442639.0, 9982791605, '서울'],
 ['2021-10', 8029655.4, 9772609520, '서울'],
 ['2021-11', 8452599.4, 11589320066, '서울']

In [18]:
df = pd.DataFrame(all_list, columns=['Date', '총거래물량', '총거래금액', '시장'])

In [20]:
df['평균거래금액'] = df['총거래금액']/df['총거래물량']

In [23]:
df = df[['Date', '총거래물량', '총거래금액', '평균거래금액', '시장']]

In [25]:
df.to_csv('data/TEMP/경락가격/대파/대파 월별 총거래(정제).csv', index=False, encoding='UTF-8')